In [1]:
import sys
sys.executable
import os
os.getcwd()
sys.path.append('devel/lib/python3/dist-packages/theconstruct_msgs/msg/')
sys.path.append('/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src')
sys.path.append('/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src/moveo_training/scripts')
sys.path.append('/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib/python3/dist-packages/openai_ros/msg')

In [2]:
sys.path

['/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL',
 '',
 '/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib/python3/dist-packages',
 '/opt/ros/noetic/lib/python3/dist-packages',
 '/home/nils/anaconda3/lib/python38.zip',
 '/home/nils/anaconda3/lib/python3.8',
 '/home/nils/anaconda3/lib/python3.8/lib-dynload',
 '/home/nils/.local/lib/python3.8/site-packages',
 '/home/nils/anaconda3/lib/python3.8/site-packages',
 '/home/nils/anaconda3/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/nils/.local/lib/python3.8/site-packages/IPython/extensions',
 '/home/nils/.ipython',
 'devel/lib/python3/dist-packages/theconstruct_msgs/msg/',
 '/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src',
 '/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/src/moveo_training/scripts',
 '/home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib/python3/dist-packages/openai_ros/msg']

In [3]:
import base64
import imageio
import IPython
import matplotlib.pyplot as plt
import os
import reverb
import tempfile
import PIL.Image

import tensorflow as tf
import moveo_inverse_kinematic
from tf_agents.environments import suite_gym
from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.agents.sac import tanh_normal_projection_network
from tf_agents.environments import suite_pybullet
from tf_agents.metrics import py_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_py_policy
from tf_agents.policies import policy_saver
from tf_agents.policies import tf_py_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.train import triggers
from tf_agents.train.utils import spec_utils
from tf_agents.train.utils import strategy_utils
from tf_agents.train.utils import train_utils

import rospy
from gym.envs.registration import register
import gym
tempdir = tempfile.gettempdir()
from tf_agents.policies import policy_saver
from tf_agents.policies import tf_py_policy
from tf_agents.utils import common

from tf_agents.metrics import tf_metrics

2021-08-26 09:03:34.296866: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2021-08-26 09:03:34.296886: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Failed to import pyassimp, see https://github.com/ros-planning/moveit/issues/86 for more info


In [4]:
num_iterations = 10000 # @param {type:"integer"}

initial_collect_steps = 1# @param {type:"integer"}
collect_steps_per_iteration = 10# @param {type:"integer"}
replay_buffer_capacity = 1000 # @param {type:"integer"}

batch_size = 256 # @param {type:"integer"}

critic_learning_rate = 3e-4 # @param {type:"number"}
actor_learning_rate = 3e-4 # @param {type:"number"}
alpha_learning_rate = 3e-4 # @param {type:"number"}
target_update_tau = 0.005 # @param {type:"number"}
target_update_period = 1 # @param {type:"number"}
gamma = 0.99 # @param {type:"number"}
reward_scale_factor = 1.0 # @param {type:"number"}

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 100# @param {type:"integer"}

num_eval_episodes = 1# @param {type:"integer"}
eval_interval = 50 # @param {type:"integer"}

policy_save_interval = 1000 # @param {type:"integer"}




In [5]:
summaries_flush_secs=10,

In [6]:

rospy.init_node("train_moveo")
env_name = 'MoveoIK-v0' # @param {type:"string"}
env = suite_gym.load('MoveoIK-v0')
env.reset()

###### Setting Group Moveit with 30 seconds wait...


[ERROR] [1629961421.347893, 28433.441000]: NOT Initialising Simulation Physics Parameters
[WARN] [1629961421.378479, 28435.011000]: Start Init ControllersConnection
[WARN] [1629961421.385777, 28435.011000]: END Init ControllersConnection
/home/nils/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0.5669771, 0.3      , 0.3      , 0.3      ], dtype=float32),
 'reward': array(0., dtype=float32),
 'step_type': array(0, dtype=int32)})

In [7]:
print('Observation Spec:')
print(env.time_step_spec().observation)
print('Action Spec:')
print(env.action_spec())

Observation Spec:
BoundedArraySpec(shape=(4,), dtype=dtype('float32'), name='observation', minimum=[ 0. -1. -1. -1.], maximum=[2. 1. 1. 1.])
Action Spec:
BoundedArraySpec(shape=(5,), dtype=dtype('float32'), name='action', minimum=-2.3559999465942383, maximum=2.3559999465942383)


In [8]:
collect_env = suite_gym.load(env_name)
eval_env = suite_gym.load(env_name)

###### Setting Group Moveit with 30 seconds wait...
###### Setting Group Moveit with 30 seconds wait...


[ERROR] [1629961422.489369, 28486.471000]: NOT Initialising Simulation Physics Parameters
[WARN] [1629961422.510162, 28487.141000]: Start Init ControllersConnection
[WARN] [1629961422.511838, 28487.141000]: END Init ControllersConnection
[ERROR] [1629961422.628200, 28489.391000]: NOT Initialising Simulation Physics Parameters
[WARN] [1629961422.649195, 28490.041000]: Start Init ControllersConnection
[WARN] [1629961422.651319, 28490.161000]: END Init ControllersConnection


In [9]:
use_gpu = True
strategy = strategy_utils.get_strategy(tpu=False, use_gpu=use_gpu)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


2021-08-26 09:03:42.714295: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-26 09:03:42.743828: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-26 09:03:42.744217: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 970 computeCapability: 5.2
coreClock: 1.2155GHz coreCount: 13 deviceMemorySize: 3.94GiB deviceMemoryBandwidth: 208.91GiB/s
2021-08-26 09:03:42.744374: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nils/Documents/Studium/Bachelorarbeit/praktischer_Teil/moveo_DRL/devel/lib:/opt/ros/noetic/lib:/opt/ros

In [10]:
observation_spec, action_spec, time_step_spec = (
      spec_utils.get_tensor_specs(collect_env))

In [11]:
with strategy.scope():
  critic_net = critic_network.CriticNetwork(
        (observation_spec, action_spec),
        observation_fc_layer_params=None,
        action_fc_layer_params=None,
        joint_fc_layer_params=critic_joint_fc_layer_params,
        kernel_initializer='glorot_uniform',
        last_kernel_initializer='glorot_uniform')

In [12]:
with strategy.scope():
  actor_net = actor_distribution_network.ActorDistributionNetwork(
      observation_spec,
      action_spec,
      fc_layer_params=actor_fc_layer_params,
      continuous_projection_net=(
          tanh_normal_projection_network.TanhNormalProjectionNetwork))

In [13]:
with strategy.scope():
  train_step = train_utils.create_train_step()

  tf_agent = sac_agent.SacAgent(
        time_step_spec,
        action_spec,
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=critic_learning_rate),
        alpha_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=alpha_learning_rate),
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        td_errors_loss_fn=tf.math.squared_difference,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        train_step_counter=train_step)

  tf_agent.initialize()

In [14]:
rate_limiter=reverb.rate_limiters.SampleToInsertRatio(samples_per_insert=3.0, min_size_to_sample=3, error_buffer=3.0)

In [15]:
table_name = 'uniform_table'
table = reverb.Table(
    table_name,
    max_size=replay_buffer_capacity,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1))

reverb_server = reverb.Server([table])

[reverb/cc/platform/tfrecord_checkpointer.cc:150]  Initializing TFRecordCheckpointer in /tmp/tmpfkw4mpjz.
[reverb/cc/platform/tfrecord_checkpointer.cc:378] Loading latest checkpoint from /tmp/tmpfkw4mpjz
[reverb/cc/platform/default/server.cc:54] Started replay server on port 18924


In [16]:
reverb_replay = reverb_replay_buffer.ReverbReplayBuffer(
    tf_agent.collect_data_spec,
    sequence_length=2,
    table_name=table_name,
    local_server=reverb_server)

In [17]:
dataset = reverb_replay.as_dataset(
      sample_batch_size=batch_size, num_steps=2).prefetch(50)
experience_dataset_fn = lambda: dataset

In [18]:
tf_eval_policy = tf_agent.policy
eval_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_eval_policy, use_tf_function=True)

In [19]:
tf_collect_policy = tf_agent.collect_policy
collect_policy = py_tf_eager_policy.PyTFEagerPolicy(
  tf_collect_policy, use_tf_function=True)

In [20]:
random_policy = random_py_policy.RandomPyPolicy(
  collect_env.time_step_spec(), collect_env.action_spec())

In [21]:
rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  reverb_replay.py_client,
  table_name,
  sequence_length=2,
  stride_length=1)

In [22]:
initial_collect_actor = actor.Actor(
  collect_env,
  random_policy,
  train_step,
  steps_per_run=initial_collect_steps,
  observers=[rb_observer])
initial_collect_actor.run()

In [23]:
env_step_metric = py_metrics.EnvironmentSteps()
collect_actor = actor.Actor(
  collect_env,
  collect_policy,
  train_step,
  steps_per_run=1,
  metrics=actor.collect_metrics(10),
  summary_dir=os.path.join(tempdir, learner.TRAIN_DIR),
  observers=[rb_observer, env_step_metric])

In [24]:
eval_actor = actor.Actor(
  eval_env,
  eval_policy,
  train_step,
  episodes_per_run=num_eval_episodes,
  metrics=actor.eval_metrics(num_eval_episodes),
  summary_dir=os.path.join(tempdir, 'eval'),
)

In [25]:
saved_model_dir = os.path.join(tempdir, learner.POLICY_SAVED_MODEL_DIR)

# Triggers to save the agent's policy checkpoints.
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        tf_agent,
        train_step,
        interval=policy_save_interval),
    triggers.StepPerSecondLogTrigger(train_step, interval=1000),
]
agent_learner = learner.Learner(
  tempdir,
  train_step,
  tf_agent,
  experience_dataset_fn,
  triggers=learning_triggers)

Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it with `scale_diag` directly instead.


2021-08-26 09:03:47.006140: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Instructions for updating:
rename to distribute_datasets_from_function


2021-08-26 09:03:47.977917: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-26 09:03:47.998423: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3192660000 Hz


In [26]:

def get_eval_metrics():
  print("Starte evaluierung")
  eval_actor.run()
  results = {}
  for metric in eval_actor.metrics:
    results[metric.name] = metric.result()
  print("Evaluierung beendete")
  return results

metrics = get_eval_metrics()

Starte evaluierung
Evaluierung beendete


In [27]:
# def get_eval_metrics_collect():
#   print("Starte evaluierung")
#   collect_actor.run()
#   results = {}
#   for metric in collect_actor.metrics:
#     results[metric.name] = metric.result()
#   print("Evaluierung beendete")
#   return results

# metrics_collect = get_eval_metrics_collect()

Starte evaluierung
Evaluierung beendete


In [28]:
def log_eval_metrics(step, metrics):
  eval_results = (', ').join(
      '{} = {:.6f}'.format(name, result) for name, result in metrics.items())
  print('step = {0}: {1}'.format(step, eval_results))

log_eval_metrics(0, metrics)

step = 0: AverageReturn = -6.510911, AverageEpisodeLength = 1.000000


In [29]:
# log_eval_metrics(0, metrics_collect)

step = 0: NumberOfEpisodes = 1.000000, EnvironmentSteps = 1.000000, AverageReturn = -4.820488, AverageEpisodeLength = 1.000000


In [30]:
try:
  %%time
except:
  pass


# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = get_eval_metrics()["AverageReturn"]
returns = [avg_return]
avg_distance = get_eval_metrics()["AverageEpisodeLength"]
distance = [avg_distance]

i =0
for _ in range(num_iterations):
  # Training.
  i +=1
  collect_actor.run()
  loss_info = agent_learner.run(iterations=1)
  
  # Evaluating.
  step = agent_learner.train_step_numpy
  if eval_interval and step % eval_interval == 0:
    metrics = get_eval_metrics()
    log_eval_metrics(step, metrics)
    returns.append(metrics["AverageReturn"])
    distance.append(metrics["AverageEpisodeLength"])
    

  if log_interval and step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))
rb_observer.close()
reverb_server.stop()



SyntaxError: invalid syntax (4077647524.py, line 15)

In [ ]:
steps = range(0,num_iterations+ 1, eval_interval)
returnscrop = returns[:151]
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim()
returns

In [ ]:
plt.plot(steps,distance)
plt.ylabel('Distance from Goal')
plt.xlabel('Step')

In [ ]:
plt.scatter(steps,returnscrop)

In [ ]:
metrics

In [ ]:
# checkpoint=common.Checkpointer(
#      saved_model_dir, max_to_keep=20
# )

   

In [ ]:
# checkpoint.initialize_or_restore()

In [ ]:
# collect_actor.run()

In [ ]:
# try:
#   %%time
# except:
#   pass


# # Reset the train step
# tf_agent.train_step_counter.assign(0)

# # Evaluate the agent's policy once before training.
# avg_return = get_eval_metrics()["AverageReturn"]
# returns = [avg_return]
# i =0
# for _ in range(num_iterations):load
#   # Training.
#   i +=1
#   collect_actor.run()
#   loss_info = agent_learner.run(iterations=1)
  
#   # Evaluating.
#   step = agent_learner.train_step_numpy
#   if eval_interval and step % eval_interval == 0:
#     metrics = get_eval_metrics()
#     log_eval_metrics(step, metrics)
#     returns.append(metrics["AverageReturn"])

#   if log_interval and step % log_interval == 0:
#     print('step = {0}: loss = {1}'.format(step, loss_info.loss.numpy()))
# rb_observer.close()
# reverb_server.stop()
